In [1]:
# General python libraires 
import os

# General DS libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

# Parallelized libraries 
import dask.dataframe as dd 

In [2]:
# Getting files 
path = 'C:/Users/redha.cherif_artefac/GitHub_perso/American_express_comp/input/'
train = dd.read_csv(os.path.join(path, 'train_data.csv'))
train_labels = dd.read_csv(os.path.join(path, 'train_labels.csv'))
test = dd.read_csv(os.path.join(path, 'test_data.csv'))

In [18]:
print(len(train))
print(len(test))

5531451
11363762


In [45]:
print(dd.compute(train.S_2.min(),train.S_2.max()))
print(dd.compute(test.S_2.min(), test.S_2.max()))
print('train: 12 months')
print('test: 18 months')

('2017-03-01', '2018-03-31')
('2018-04-01', '2019-10-31')
train: 12 months
test: 18 months


In [15]:
train_cust_ID = train.customer_ID.unique().compute()
train_labels_custID = train_labels.customer_ID.unique().compute()
test_cust_ID = test.customer_ID.unique().compute()

In [21]:
print(f'Number of customers in train : {len(train_cust_ID)}')
print(f'Number of customers in test : {len(test_cust_ID)}')

Number of customers in train : 458913
Number of customers in test : 924621


In [53]:
train_test_join = pd.merge(train_cust_ID,test_cust_ID,how='inner')
train_trainLabel_join = pd.merge(train_cust_ID,train_labels_custID,how='inner')
print(f'Number of common customers in the train and the test : {len(train_test_join)}')
print(f'Number of common customers in the train and the train_labels : {len(train_trainLabel_join)}')
print('')
print('Therefore, all customers in the test are different from the ones in the train ! No leakage possible at first glance.')
print('The customers in the train_labels are the same than in the train')

Number of common customers in the train and the test : 0
Number of common customers in the train and the train_labels : 458913

Therefore, all customers in the test are different from the ones in the train ! No leakage possible at first glance.
The customers in the train_labels are the same than in the train


In [4]:
cat_feats = train[['customer_ID','B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]
cat_feats.head()

,customer_ID,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,D_68
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.0,2.0,1.0,0.0,4.0,0.0,1.0,CR,O,NaN,6.0


In [83]:
print(f'Unique values for B_30: {cat_feats.B_30.unique().compute()}')
print(f'Unique values for B_38: {cat_feats.B_38.unique().compute()}')
print(f'Unique values for D_114: {cat_feats.D_114.unique().compute()}')
print(f'Unique values for D_116: {cat_feats.D_116.unique().compute()}')
print(f'Unique values for D_117: {cat_feats.D_117.unique().compute()}')
print(f'Unique values for D_120: {cat_feats.D_120.unique().compute()}')
print(f'Unique values for D_126: {cat_feats.D_126.unique().compute()}')
print(f'Unique values for D_63: {cat_feats.D_63.unique().compute()}')
print(f'Unique values for D_64: {cat_feats.D_64.unique().compute()}')
print(f'Unique values for D_66: {cat_feats.D_66.unique().compute()}')
print(f'Unique values for D_68: {cat_feats.D_68.unique().compute()}')

Unique values for B_30: 0    0.0
1    2.0
2    1.0
3    NaN
Name: B_30, dtype: float64
Unique values for B_38: 0    2.0
1    1.0
2    3.0
3    5.0
4    6.0
5    7.0
6    4.0
7    NaN
Name: B_38, dtype: float64
Unique values for D_114: 0    1.0
1    0.0
2    NaN
Name: D_114, dtype: float64
Unique values for D_116: 0    0.0
1    NaN
2    1.0
Name: D_116, dtype: float64
Unique values for D_117: 0    4.0
1   -1.0
2    6.0
3    2.0
4    1.0
5    NaN
6    3.0
7    5.0
Name: D_117, dtype: float64
Unique values for D_120: 0    0.0
1    1.0
2    NaN
Name: D_120, dtype: float64
Unique values for D_126: 0    1.0
1    NaN
2    0.0
3   -1.0
Name: D_126, dtype: float64
Unique values for D_63: 0    CR
1    CO
2    CL
3    XZ
4    XM
5    XL
Name: D_63, dtype: object
Unique values for D_64: 0      O
1      R
2    NaN
3      U
4     -1
Name: D_64, dtype: object
Unique values for D_66: 0    NaN
1    1.0
2    0.0
Name: D_66, dtype: float64
Unique values for D_68: 0    6.0
1    2.0
2    3.0
3    NaN
4    

In [16]:
print('The number of D columns : 95 with 9 categorical columns => going from D_39 to D_145 with the lack of D_40, 57, 67, 85, 90, 95, 97, 98, 99, 100 and 101 (11 columns)')
print('The number of S columns : 22 numerical columns => lack of 5 columns \n (S_2, S_3, S_5, S_6, S_7, S_8, S_9, S_11, S_12, S_13, S_15, S_16, S_17, S_18, S_19, S_20, S_22, S_23, S_24, S_25, S_26, S_27')
print('The number of P columns : 3 numerical columns => P_2, P_3 and P_4 with the lack of P_1. P_3 looks to be a normal distribution')
print('The number of B columns : 40 with 2 categorical columns => going from B_1 to B_42 with the lack of B_34 and B_35')
print('The number of R columns : 3 numerical columns => R_2, R_3 and R_4 with the lack of R_1')

The number of D columns : 95 with 9 categorical columns => going from D_39 to D_145 with the lack of D_40, 57, 67, 85, 90, 95, 97, 98, 99, 100 and 101 (11 columns)
The number of S columns : 22 numerical columns => lack of 5 columns 
 (S_2, S_3, S_5, S_6, S_7, S_8, S_9, S_11, S_12, S_13, S_15, S_16, S_17, S_18, S_19, S_20, S_22, S_23, S_24, S_25, S_26, S_27
The number of P columns : 3 numerical columns => P_2, P_3 and P_4 with the lack of P_1. P_3 looks to be a normal distribution
The number of B columns : 40 with 2 categorical columns => going from B_1 to B_42 with the lack of B_34 and B_35
The number of R columns : 3 numerical columns => R_2, R_3 and R_4 with the lack of R_1


In [12]:
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [5]:
train_trainLabel_join = train.merge(train_labels,how='left',on='customer_ID')
catfeats_trainLabel_join = cat_feats.merge(train_labels,how='left',on='customer_ID')

In [13]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['B_30'].unique().compute()

0    1.0
1    0.0
2    2.0
3    NaN
Name: B_30, dtype: float64

In [14]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['B_38'].unique().compute()

0    5.0
1    6.0
2    7.0
3    1.0
4    4.0
5    3.0
6    2.0
7    NaN
Name: B_38, dtype: float64

In [15]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_114'].unique().compute()

0    0.0
1    1.0
2    NaN
Name: D_114, dtype: float64

In [16]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_116'].unique().compute()

0    0.0
1    NaN
2    1.0
Name: D_116, dtype: float64

In [17]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_117'].unique().compute()

0   -1.0
1    4.0
2    NaN
3    1.0
4    2.0
5    3.0
6    5.0
7    6.0
Name: D_117, dtype: float64

In [18]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_120'].unique().compute()

0    0.0
1    NaN
2    1.0
Name: D_120, dtype: float64

In [19]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_126'].unique().compute()

0    1.0
1    0.0
2   -1.0
3    NaN
Name: D_126, dtype: float64

In [41]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_63'].unique().compute()

0    CO
1    CL
2    CR
3    XM
4    XL
5    XZ
Name: D_63, dtype: object

In [20]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_64'].unique().compute()

0      U
1      O
2    NaN
3      R
4     -1
Name: D_64, dtype: object

In [21]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_66'].unique().compute()

0    1.0
1    NaN
2    0.0
Name: D_66, dtype: float64

In [22]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1]['D_68'].unique().compute()

0    3.0
1    6.0
2    5.0
3    NaN
4    4.0
5    1.0
6    2.0
7    0.0
Name: D_68, dtype: float64

In [9]:
catfeats_trainLabel_join[catfeats_trainLabel_join['target']==1].head()

,customer_ID,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_66,D_68,target
104,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,1.0,5.0,0.0,0.0,-1.0,0.0,1.0,CO,U,1.0,3.0,1
105,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,1.0,5.0,0.0,0.0,-1.0,0.0,1.0,CO,U,1.0,3.0,1
106,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,1.0,6.0,0.0,0.0,-1.0,0.0,1.0,CO,U,1.0,6.0,1
107,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,1.0,7.0,0.0,0.0,-1.0,0.0,1.0,CO,U,1.0,6.0,1
108,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,1.0,7.0,0.0,0.0,-1.0,0.0,1.0,CO,U,1.0,6.0,1


In [33]:
# Statistics for P and R columns for customers with default payments 
train_trainLabel_join[train_trainLabel_join['target']==1][['P_2','P_3','P_4','R_2','R_3','R_4']].describe().compute()

,P_2,P_3,P_4,R_2,R_3,R_4
count,1.366093e+06,1.244039e+06,1.377869e+06,1.377869e+06,1.377869e+06,1.377869e+06
mean,3.968602e-01,5.267698e-01,2.834218e-01,1.399179e-01,2.256505e-01,9.253445e-02
std,1.945577e-01,2.253384e-01,4.379168e-01,3.416449e-01,3.161011e-01,2.826330e-01
min,-4.589548e-01,-1.298901e+00,1.009896e-08,2.810394e-09,2.238322e-09,1.618992e-09
25%,3.188087e-01,4.506216e-01,3.881625e-03,3.093204e-03,8.623292e-03,2.923374e-03
50%,4.320343e-01,5.765630e-01,7.614955e-03,6.060022e-03,2.012764e-01,5.677573e-03
75%,5.462155e-01,6.765356e-01,9.587939e-01,9.051380e-03,4.023662e-01,8.464747e-03
max,1.009979e+00,2.139391e+00,1.269390e+00,1.010000e+00,1.160260e+01,1.010000e+00


In [30]:
# # Statistics for P and R columns for customers without default payments 
train_trainLabel_join[train_trainLabel_join['target']==0][['P_2','P_3','P_4','R_2','R_3','R_4']].describe().compute()

,P_2,P_3,P_4,R_2,R_3,R_4
count,4.119373e+06,3.985920e+06,4.153582e+06,4.153582e+06,4.153582e+06,4.153582e+06
mean,7.423823e-01,6.245465e-01,9.765245e-02,1.686603e-02,8.605611e-02,1.085900e-02
std,1.934366e-01,1.419689e-01,2.831512e-01,1.083154e-01,1.449745e-01,7.637867e-02
min,-4.210819e-01,-1.519690e+00,2.668066e-09,8.265748e-10,3.823338e-09,4.024685e-10
25%,6.412804e-01,5.740687e-01,2.890483e-03,2.622405e-03,4.465579e-03,2.594037e-03
50%,8.061569e-01,6.313309e-01,5.677832e-03,5.166691e-03,8.865137e-03,5.134510e-03
75%,9.052483e-01,6.965521e-01,8.501004e-03,7.669449e-03,1.084215e-01,7.629766e-03
max,1.010000e+00,2.428051e+00,1.221814e+00,1.009999e+00,6.101872e+00,1.009999e+00


In [34]:
train_trainLabel_join[train_trainLabel_join['target']==1][['P_2','P_3','P_4','R_2','R_3','R_4']].describe().compute() - train_trainLabel_join[train_trainLabel_join['target']==0][['P_2','P_3','P_4','R_2','R_3','R_4']].describe().compute()

,P_2,P_3,P_4,R_2,R_3,R_4
count,-2.753280e+06,-2.741881e+06,-2.775713e+06,-2.775713e+06,-2.775713e+06,-2.775713e+06
mean,-3.455222e-01,-9.777671e-02,1.857693e-01,1.230519e-01,1.395944e-01,8.167545e-02
std,1.121170e-03,8.336954e-02,1.547655e-01,2.333296e-01,1.711266e-01,2.062543e-01
min,-3.787289e-02,2.207884e-01,7.430891e-09,1.983819e-09,-1.585016e-09,1.216523e-09
25%,-3.224717e-01,-1.234472e-01,9.911421e-04,4.707995e-04,4.157712e-03,3.293373e-04
50%,-3.741226e-01,-5.476796e-02,1.937123e-03,8.933307e-04,1.924112e-01,5.430634e-04
75%,-3.590327e-01,-2.001653e-02,9.502929e-01,1.381931e-03,2.939447e-01,8.349806e-04
max,-2.099481e-05,-2.886597e-01,4.757611e-02,4.719225e-07,5.500729e+00,7.205739e-07


In [3]:
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [10]:
S_cols_def_pay = train_trainLabel_join[train_trainLabel_join['target']==1][['S_2','S_3','S_5','S_6','S_7','S_8','S_9','S_11','S_12','S_13','S_15','S_16','S_17','S_18','S_19','S_20','S_22','S_23','S_24','S_25','S_26','S_27']]
S_cols_non_def_pay = train_trainLabel_join[train_trainLabel_join['target']==0][['S_2','S_3','S_5','S_6','S_7','S_8','S_9','S_11','S_12','S_13','S_15','S_16','S_17','S_18','S_19','S_20','S_22','S_23','S_24','S_25','S_26','S_27']]

In [11]:
S_cols_def_pay.describe().compute() - S_cols_non_def_pay.describe().compute()

,S_3,S_5,S_6,S_7,S_8,S_9,S_11,S_12,S_13,S_15,...,S_17,S_18,S_19,S_20,S_22,S_23,S_24,S_25,S_26,S_27
count,-1.972321e+06,-2.775713e+06,-2.775713e+06,-1.972321e+06,-2.775713e+06,-1.437102e+06,-2.775713e+06,-2.775713e+06,-2.775713e+06,-2.775713e+06,...,-2.775713e+06,-2.775713e+06,-2.775713e+06,-2.775713e+06,-2.761711e+06,-2.775734e+06,-2.762018e+06,-2.779760e+06,-2.775247e+06,-1.850282e+06
mean,1.389090e-01,4.216301e-02,-9.760135e-02,1.535118e-01,-1.079924e-01,3.410003e-02,-3.771261e-02,4.348149e-03,-7.280447e-02,8.711571e-02,...,1.713506e-02,-3.720856e-03,6.848923e-05,3.415558e-02,1.470367e-01,1.123387e-01,1.816579e-01,-1.180682e-01,-4.532594e-02,-1.705532e-02
std,9.455598e-02,3.430789e-02,-6.853650e-02,8.558425e-02,-8.769212e-02,8.495533e-02,-1.663748e-02,1.377511e-01,-5.368754e-02,4.634428e-02,...,8.913913e-02,-1.119509e-02,4.633629e-04,1.285200e-01,-2.293524e-01,6.941359e-01,-1.960231e-01,1.963826e-01,-1.997615e-01,1.180842e-01
min,1.601017e-01,4.703833e-08,-4.768716e-09,1.166325e-01,5.670660e-09,-1.636389e-07,1.596671e-09,5.809359e-04,-9.506126e-09,4.497312e-06,...,5.261058e-09,3.313988e-09,3.238832e-08,6.545888e-09,2.492526e-03,1.957776e+02,1.637482e-04,-6.150066e-07,1.262052e-08,4.065132e-08
25%,4.916833e-02,4.395137e-03,-3.080030e-04,7.466106e-02,8.536598e-04,1.793040e-03,-1.936296e-03,-4.282358e-05,-5.174210e-04,9.958056e-02,...,5.002623e-05,1.101735e-04,1.121559e-04,1.556140e-04,6.047972e-01,5.334569e-04,8.047978e-01,-2.090529e-03,-3.825537e-04,-3.010735e-03
50%,1.125280e-01,3.024464e-02,-6.816162e-04,1.877427e-01,-1.491869e-02,8.790194e-03,-2.735007e-03,-4.321301e-04,-2.754640e-01,9.948735e-02,...,5.941171e-05,5.569268e-05,5.220950e-05,3.105062e-04,2.752677e-02,1.703949e-03,2.284123e-02,-1.186708e-03,-8.551584e-04,-7.895082e-02
75%,2.477159e-01,6.739471e-02,-9.918136e-01,2.331553e-01,-2.541719e-01,4.198703e-02,-3.847187e-02,-2.368736e-02,-4.632271e-03,4.226166e-03,...,-7.407075e-05,5.089062e-05,9.692899e-05,3.634239e-04,1.992541e-02,1.867102e-01,1.431987e-02,-2.777042e-04,-2.668999e-02,4.889954e-02
max,1.044930e+00,-1.292670e+02,-4.771698e-08,7.361636e-01,-1.335655e-05,6.749757e-05,-7.972740e-01,8.238470e+01,-4.701835e-07,-5.171077e-04,...,-2.051110e-05,5.645362e-08,8.214087e-01,2.239098e-07,-1.661890e-06,-2.026756e+02,-1.269106e-03,5.334403e-01,-4.119444e+01,1.364593e-01


## Missing values

In [14]:
train_missing_values = train.isnull().sum().compute() / len(train)

In [29]:
pd.set_option('display.max_rows', None)
train_missing_values = train_missing_values * 100 
train_missing_values[train_missing_values != 0].sort_values(ascending=False)

D_87     99.930127
D_88     99.891457
D_108    99.476846
D_110    99.433530
D_111    99.433530
B_39     99.391986
D_73     98.990211
B_42     98.707789
D_134    96.480146
D_137    96.480146
D_138    96.480146
D_135    96.480146
D_136    96.480146
R_9      94.349891
B_29     93.104594
D_106    90.213255
D_132    90.191055
D_49     90.137597
R_26     88.984717
D_76     88.746226
D_66     88.730733
D_42     85.694278
D_142    82.926577
D_53     73.842921
D_82     73.373406
D_50     56.809723
B_17     56.722874
D_105    54.622756
D_56     54.071581
S_9      53.035686
D_77     45.447605
D_43     29.981211
S_27     25.326718
D_46     21.905627
S_3      18.449843
S_7      18.449843
D_62     13.706367
D_48     12.993426
D_61     10.811847
P_3       5.450505
D_44      4.959259
D_78      4.959259
D_64      3.931012
D_68      3.914036
D_83      3.515768
D_69      3.515768
D_55      3.340950
D_113     3.194749
D_114     3.194749
D_115     3.194749
D_116     3.194749
D_117     3.194749
D_119     3.

In [32]:
test_missing_values = test.isnull().sum().compute() / len(test)
test_missing_values = test_missing_values * 100 
test_missing_values[test_missing_values != 0].sort_values(ascending=False)

D_87     99.903447
D_88     99.900869
D_108    99.403587
D_110    98.976624
D_111    98.976624
B_39     98.789213
D_73     98.683737
B_42     98.228421
D_134    96.814039
D_136    96.814039
D_137    96.814039
D_138    96.814039
D_135    96.814039
R_9      94.936527
D_76     89.519545
D_66     88.121592
D_106    88.096618
D_132    88.077505
D_49     88.059412
D_42     88.053402
R_26     86.003350
D_142    82.692395
B_29     75.038205
D_82     74.382242
D_53     73.734165
D_50     57.047534
B_17     53.909788
D_105    53.399578
D_56     50.271275
D_77     49.040318
S_9      35.070903
D_43     30.030777
S_27     22.002177
D_46     20.214723
D_62     16.645139
S_3      15.320323
S_7      15.320323
D_48     12.734656
D_61     10.913296
D_55      6.538090
R_27      5.838771
D_78      4.735800
D_44      4.735800
P_3       3.947452
D_91      3.875451
D_64      3.324938
D_68      3.066731
D_83      2.536766
D_69      2.536766
D_122     2.501293
D_124     2.501293
D_113     2.501293
D_115     2.

In [ ]:
# Delete columns with more than 45% of missing values 
cols_train_to_delete = ['D_87','D_88','D_108','D_110','D_111','B_39','D_73','B_42','D_134','D_137','D_138','D_135','D_136','R_9','B_29','D_106','D_132','D_49','R_26','D_76','D_66','D_42','D_142','D_53','D_82','D_50','B_17','D_105','D_56','S_9','D_77']
train = train.drop(labels=cols_train_to_delete,axis=1)

In [21]:
# Delete columns with more than 49% of missing values 
cols_test_to_delete = ['D_87','D_88','D_108','D_110','D_111','B_39','D_73','B_42','D_134','D_137','D_138','D_135','D_136','R_9','D_76','D_66','D_106','D_132','D_49','D_42','R_26','D_142','B_29','D_82','D_53','D_50','B_17','D_105','D_56', 'D_77']
test = test.drop(labels=cols_test_to_delete,axis=1)

In [25]:
pd.set_option('display.max_rows', None)
train_missing_values = train.isnull().sum().compute() / len(train) * 100
train_missing_values[train_missing_values != 0].sort_values(ascending=False)

D_43     29.981211
S_27     25.326718
D_46     21.905627
S_3      18.449843
S_7      18.449843
D_62     13.706367
D_48     12.993426
D_61     10.811847
P_3       5.450505
D_44      4.959259
D_78      4.959259
D_64      3.931012
D_68      3.914036
D_83      3.515768
D_69      3.515768
D_55      3.340950
D_122     3.194749
D_125     3.194749
D_116     3.194749
D_123     3.194749
D_120     3.194749
D_119     3.194749
D_118     3.194749
D_121     3.194749
D_115     3.194749
D_124     3.194749
D_114     3.194749
D_113     3.194749
D_117     3.194749
D_91      2.842220
R_27      2.326749
D_126     2.111851
D_59      1.929421
D_107     1.835829
D_104     1.835829
D_145     1.835829
D_128     1.835829
D_129     1.835829
D_130     1.835829
D_131     1.835829
D_139     1.835829
D_141     1.835829
D_143     1.835829
D_103     1.835829
D_70      1.715951
D_79      1.372859
B_13      0.895226
P_2       0.831337
D_133     0.772239
D_144     0.736281
D_102     0.734979
D_140     0.734563
D_89      0.

In [26]:
test_missing_values = test.isnull().sum().compute() / len(test) * 100
test_missing_values[test_missing_values != 0].sort_values(ascending=False)

S_9      35.070903
D_43     30.030777
S_27     22.002177
D_46     20.214723
D_62     16.645139
S_7      15.320323
S_3      15.320323
D_48     12.734656
D_61     10.913296
D_55      6.538090
R_27      5.838771
D_78      4.735800
D_44      4.735800
P_3       3.947452
D_91      3.875451
D_64      3.324938
D_68      3.066731
D_83      2.536766
D_69      2.536766
D_121     2.501293
D_124     2.501293
D_120     2.501293
D_119     2.501293
D_118     2.501293
D_117     2.501293
D_122     2.501293
D_115     2.501293
D_123     2.501293
D_114     2.501293
D_125     2.501293
D_113     2.501293
D_116     2.501293
D_59      1.464436
D_70      1.215971
B_8       0.918182
D_107     0.859750
D_104     0.859750
D_103     0.859750
D_145     0.859750
D_128     0.859750
D_129     0.859750
D_130     0.859750
D_131     0.859750
D_139     0.859750
D_141     0.859750
D_143     0.859750
D_79      0.800325
D_126     0.725121
S_22      0.538624
S_24      0.529367
P_2       0.526921
B_13      0.514425
D_74      0.

In [60]:
train[train['D_43']==train.D_43.max().compute()]

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,B_6,B_7,B_8,D_51,B_9,R_3,D_52,P_3,B_10,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,B_13,R_5,D_58,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_18,B_19,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,P_4,D_74,D_75,B_24,R_7,B_25,B_26,D_78,D_79,R_8,S_16,D_80,R_10,R_11,B_27,D_81,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_30,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_107,B_36,B_37,R_27,B_38,D_109,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145
npartitions=256,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...